In [1]:
import requests
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [117]:
#Make Sure We Can Stil Access the Site
url = 'https://www.basketball-reference.com/leagues/NBA_1999_transactions.html'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
#Response[200]
response

In [172]:
test_df = pd.DataFrame(columns=['Date','Team','Player'])

In [173]:
def getAgentDict(year):
    
    #Get URL and get all information from that page
    url = f'https://www.basketball-reference.com/leagues/NBA_{year}_transactions.html'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #Create a dictionary where keys will be dates and values will be all events on that day
    transactions = dict()
    
    #For each date where something happened
    for day in soup.find('ul',class_='page_index'):
    
        #If it found a date
        if day.find('span') != -1:

            #Grab the date and set that as key and bs4.element.Tag of Transactions as values
            date = day.find('span').get_text()
            transactions[date] = day
        else:
            date = None   
    
    #For each date that events went on        
    for date in transactions:    
        
        temp = []
        trans = transactions[date]

        #Get all transactions for that day and events for that day
        all_trans = trans.find_all('p')

        #For each transaction/event
        for t in all_trans:

            #If transaction/event is for free agent, grab the team name and player involved and add to list temp for that day
            if " as a free agent." in t.get_text():
                deal = t.find_all('a')
                for it in deal:
                    i = it.get_text()
                    temp.append(i)

        transactions[date] = temp

    for day in transactions:
        trans = transactions[day]
        
        #Transactions values currently a list where transactions 1,2,... go [team1,player1,team2,player2,.....]
        #So even grabs the team and odd the free agent player
        if len(trans) != 0:
            for n in range(len(trans)-1):
                if n % 2 == 0:
                    global test_df
                    test_df = test_df.append({'Date':day,'Team':trans[n],'Player':trans[n+1]},ignore_index=True)

In [176]:
#Get free agent transactions from 1999-2023 seasons
for y in range(1999,2024):
    getAgentDict(y)
    time.sleep(4)

In [177]:
test_df

,Date,Team,Player
0,"January 21, 1999",Utah Jazz,Shawnelle Scott
1,"January 21, 1999",Sacramento Kings,Isaac Fontaine
2,"January 21, 1999",New Jersey Nets,Steve Goodrich
3,"January 21, 1999",Orlando Magic,Jonathan Kerner
4,"January 21, 1999",Atlanta Hawks,Jeff Sheppard
...,...,...,...
2334,"December 4, 2020",Oklahoma City Thunder,Frank Jackson
2335,"December 9, 2020",Utah Jazz,Shaquille Harrison
2336,"January 7, 2021",New York Knicks,Taj Gibson
2337,"April 11, 2021",Brooklyn Nets,Alize Johnson


In [178]:
test_df.to_csv('C:\\Users\\finnr\\Downloads\\Zhao McIntire Research\\ScrapingFreeAgents\\freeagents.csv')